# Certifiably Optimal RulE ListS (CORELS)

Produce a simple rule list for a dataset with binary features. A chain of if-then statements is produced to predict the labels with the highest accuracy.

In [49]:
import xarray as xr
import netCDF4
from pathlib import Path
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import sys
import gc
import geopandas as gpd
import rioxarray as rxr
from shapely.geometry import box
from affine import Affine
import sys
import seaborn as sns
import rasterio
import os
from collections import defaultdict
import random
from methods.corels import Corels
from utils.utils import binarize_data

In [2]:
import warnings
warnings.filterwarnings('ignore')

Upload UC3 data: 

In [3]:
df = pd.read_csv(r"/Users/Michele/Desktop/ISP/projects/deepcube/uc3/data/greece_pixel_dataset.csv")

In [4]:
features = [
 'Fpar_500m',
 'Lai_500m',
 'LST_Day_1km',
 'LST_Night_1km',
 '1 km 16 days NDVI',
 '1 km 16 days EVI',
 'ET_500m',
 'LE_500m',
 'PET_500m',
 'PLE_500m',
 'era5_max_u10',
 'era5_max_v10',
 'era5_max_t2m',
 'era5_max_tp',
 'era5_min_u10',
 'era5_min_v10',
 'era5_min_t2m',
 'era5_min_tp'
]

coordinates = ['x', 'y']

static_features = [ 
 'dem_mean',
 'dem_std',
 'aspect_mean',
 'aspect_std',
 'slope_mean',
 'slope_std',
 'roads_density_2020',
 'population_density',
 'clc'
]

target = 'burned_areas'

Data cleaning and splitting: 

In [5]:
nan_fillvalue = -9999
df = df.fillna(nan_fillvalue)
time_split = int(df.time.max()*0.85)
train_df=df[df['time']<time_split]
test_df=df[df['time']>=time_split]

Fit a Random Forest and plot results: 

In [34]:
features_to_exclude = [
 'Fpar_500m',
 'Lai_500m',
#  'LST_Day_1km',
#  'LST_Night_1km',
#  '1 km 16 days NDVI',
#  '1 km 16 days EVI',
 'ET_500m',
 'LE_500m',
 'PET_500m',
 'PLE_500m',
#  'era5_max_u10',
#  'era5_max_v10',
#  'era5_max_t2m',
#  'era5_max_tp',
 'era5_min_u10',
 'era5_min_v10',
 'era5_min_t2m',
 'era5_min_tp',
#  'dem_mean',
 'dem_std',
#  'aspect_mean',
 'aspect_std',
#  'slope_mean',
#  'slope_std',
#  'roads_density_2020',
#  'population_density',
#  'clc'
]
features_filtered = [x for x in features + static_features if x not in features_to_exclude]

X_train, X_test = train_df[features_filtered], test_df[features_filtered]
y_train, y_test = train_df[target], test_df[target]

Let us fit the PDP method for plotting the univariate relation between the model and one of the predictors we specify:

In [35]:
features_filtered

['LST_Day_1km',
 'LST_Night_1km',
 '1 km 16 days NDVI',
 '1 km 16 days EVI',
 'era5_max_u10',
 'era5_max_v10',
 'era5_max_t2m',
 'era5_max_tp',
 'dem_mean',
 'aspect_mean',
 'slope_mean',
 'slope_std',
 'roads_density_2020',
 'population_density',
 'clc']

Let us produce a list of rules based on a binarized version of the data:

In [36]:
nan_idx = []
for f in range(X_train.shape[1]):
    nan_idx += list(np.where(np.array(X_train)[:,f]== nan_fillvalue)[0])

nan_idx = list(np.unique(nan_idx))
index_filtered = list(set(range(X_train.shape[0]))-set(nan_idx))
X_clean = np.array(X_train)[index_filtered,:]
y_clean = np.array(y_train)[index_filtered]

In [37]:
config = {"X": X_clean, 
         "y": y_clean, 
         "features": features_filtered}

In [38]:
c = Corels(config)
clf = c.fit()

In [41]:
c.plot(clf)

RULELIST:
if [LST_Night_1km >= 292.154 && not era5_max_v10 >= -0.666]:
  Fire Risk = True
else if [LST_Day_1km >= 305.598 && era5_max_t2m >= 301.047]:
  Fire Risk = True
else 
  Fire Risk = False


In [40]:
c.run_all()

Rule list for Fire Risk:
RULELIST:
if [LST_Night_1km >= 292.154 && not era5_max_v10 >= -0.666]:
  Fire Risk = True
else if [LST_Day_1km >= 305.598 && era5_max_t2m >= 301.047]:
  Fire Risk = True
else 
  Fire Risk = False


In [54]:
print("Train set accuracy = ", clf.score(X_clean, y_clean))

Train set accuracy =  0.8406943528894748


Let us see what is the accuracy over (unseen) test data:

In [46]:
nan_idx = []
for f in range(X_test.shape[1]):
    nan_idx += list(np.where(np.array(X_test)[:,f]== nan_fillvalue)[0])

nan_idx = list(np.unique(nan_idx))
index_filtered = list(set(range(X_test.shape[0]))-set(nan_idx))
Xt_clean = np.array(X_test)[index_filtered,:]
yt_clean = np.array(y_test)[index_filtered]

In [50]:
_, Xt_bin = binarize_data(Xt_clean, features_filtered)

In [53]:
print("Test set accuracy = ", clf.score(Xt_bin, yt_clean))

Test set accuracy =  0.71045197740113
